In [1]:

from google.colab import drive
drive.mount('/content/gdrive')  #mounting


Mounted at /content/gdrive


In [2]:
import random
import os
from PIL import Image
import torch
import torchvision
import numpy as np
import torchvision.transforms as T

In [3]:
%cd '/content/gdrive/MyDrive/DEEPLOBE'

/content/gdrive/MyDrive/DEEPLOBE


In [5]:
# !unzip data.zip

In [8]:
#!unzip deeplobe-prod-main.zip

In [9]:
ls

data/  data.zip  deeplobe-prod-main/  deeplobe-prod-main.zip


In [10]:
class UserDataset(torch.utils.data.Dataset):
    """
    Custom class inheriting from Pytorch's Dataset utility class
    that allows applying custom transformations on user-datasets

    It returns transformed images and masks in an iterator object
    that can be indexed according to the batch sizes
    in the data loading phase for passing to model
    """

    def __init__(
        self,
        #img_files: list,
        #path_lbl: str,
        class_dict: dict = None,
        img_transform=None,
        mask_transform=None,
        request_uuid=None,
    ):

        self.img_files = '/content/gdrive/MyDrive/DEEPLOBE/data/Images'
        self.path_lbl = '/content/gdrive/MyDrive/DEEPLOBE/data/Masks'  ##### changes here ##### mask file path
        self.img_transform = img_transform
        self.mask_transform = mask_transform
        self.class_dict = class_dict
        self.request_uuid = request_uuid

    # helper function to convert rbg mask to label encoded masks
    def mask_to_class(self, mask):
        """
        arguments: mask -  colored mask of the image in torch.tensor format
                            mathcing the image shape
        returns: mask - torch.tensor where RGB channels are replaced by class labels

        Convert mask with RGB channels into label-encoded mask where
        each RGB channel (which looks like (0,255,0) or (255,0,0) will be
        converted into a relavent label like 1 or 2)
        """
        target = mask.permute(1, 2, 0).contiguous()
        h, w = target.shape[0], target.shape[1]
        masks = torch.zeros(h, w, dtype=torch.long)
        target = target.permute(2, 0, 1).contiguous()
        mapping = self.class_dict

        for k in mapping:
            idx = target == torch.tensor(k, dtype=torch.uint8).unsqueeze(1).unsqueeze(2)
            validx = idx.sum(0) == 3
            masks[validx] = torch.tensor(mapping[k], dtype=torch.long)
        return masks

    def __getitem__(self, idx: int):
        image_file = self.img_files[idx]
        image_filename = os.path.split(image_file)[1]  ###### changes here ####
        mask_file = os.path.join(
            self.path_lbl, image_filename.split(".")[0] + ".png"
        )  ###### changes here ####
        from deeplobedj.web.views import s3_images_semantic, s3_masks_semantic

        image_file = s3_images_semantic(image_file, request_uuid=self.request_uuid)
        mask_file = s3_masks_semantic(mask_file, request_uuid=self.request_uuid)
        # print(image, "___________76")
        # image = Image.open(image)
        # mask = Image.open(mask)
        # print(image, "________80___________-")

        image = Image.open(image_file)
        mask = Image.open(mask_file)

        # apply tranforms
        if self.img_transform:
            image = self.img_transform(image)

        if self.mask_transform:
            mask = self.mask_transform(mask)

        # encode RGB mask to class_labels
        mask = self.mask_to_class(mask)

        return image, mask

    def __len__(self):  # return count of sample we have
        return len(self.img_files)

In [ ]:
class PreProcessing:
    def __init__(self, batch_size: int = 4, valid_pct: float = 0.2):
        self.batch_size = batch_size
        self.valid_pct = valid_pct
        self.uploads = r"./Uploads"
        self.path = os.path.join(self.uploads, "segmentation", "semantic_segmentation")
        self.path_lbl = os.path.join(self.path, "Masks")
        self.mask_definitions = os.path.join(self.path, "mask_definitions.json")
        self.extensions = ["jpg", "jpeg", "png", "PNG"]

        self.transforms = {
            "img_transform": T.Compose(
                [
                    T.Resize(128),
                    T.CenterCrop(120),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ]
            ),
            "mask_transform": T.Compose(
                [T.Resize(128), T.CenterCrop(120), T.ToTensor()]
            ),
        }

    def train_test_split(self, img_files, mask_files):

        dataset_size = len(img_files)
        valid_size = int(dataset_size * self.valid_pct)

        # images
        valid_imgs = random.sample(img_files, valid_size)
        train_imgs = [img for img in img_files if img not in valid_imgs]

        # corresponding masks
        valid_img_names = [os.path.split(img)[1].split(".")[0] for img in valid_imgs]
        valid_masks = [
            mask
            for mask in mask_files
            if os.path.split(mask)[1].split(".")[0] in valid_img_names
        ]
        train_masks = [mask for mask in mask_files if mask not in valid_masks]

        return train_imgs, valid_imgs, train_masks, valid_masks

    def get_class_dict(self, request_uuid=None) -> dict:
        # Extract and return label names and color-codes from mask_definitions json
        from deeplobedj.web.views import semantic_masks_download

        semantic_masks_download("download", request_uuid)
        with open(self.mask_definitions) as md:
            mask_def = eval(md.read())

        mask_info = {}
        label_dict = {}
        img_files = []
        mask_files = []

        for image in mask_def["masks"]:
            img_files.append(os.path.join(self.path, image))
            mask = mask_def["masks"][image]["mask"]
            mask_files.append(os.path.join(self.path, mask))
            mask = mask.lstrip("masks/")
            colors = mask_def["masks"][image]["color_categories"]
            rgb = colors.keys()
            categories = [i["super_category"] for i in colors.values()]
            color_map = dict(zip(rgb, categories))
            mask_info.update({mask: color_map})

        for v in mask_info.values():
            label_dict.update(v.items())

        # create an rgb dict with colors as keys and labels as values
        label_dict["(0, 0, 0)"] = "background"
        class_dict = {eval(v): k for k, v in enumerate(sorted(label_dict.keys()))}

        return class_dict, img_files, mask_files

